In [17]:
import sys
sys.path.append("../..") 

from custom_scripts import database

In [18]:
### Table of unique carrier identifiers present in the test samples
carriers = database.query(""" 
                            SELECT op_unique_carrier, COUNT(*)
                                FROM flights_test
                                GROUP BY op_unique_carrier;
                            """)
carriers.to_csv('../../data/preprocessing/unique_carriers.csv', index=False)
carriers.head(1)

,op_unique_carrier,count
0,9E,23088


In [19]:
### Table of flight numbers present in the test samples, and a count of their frequency
test_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*) 
                                    FROM flights_test
                                    GROUP BY op_carrier_fl_num;
                                """)
test_flight_numbers.to_csv('../../data/preprocessing/test_flight_numbers.csv', index=False)
test_flight_numbers.head(1)

,op_carrier_fl_num,count
0,6114,64


In [20]:
### Table of flight numbers present in the samples available for training, and a count of their frequency
past_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*)
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY op_carrier_fl_num;
                                """)
past_flight_numbers.to_csv('../../data/preprocessing/past_flight_numbers.csv', index=False)
past_flight_numbers.head(1)


,op_carrier_fl_num,count
0,1,308


In [21]:
### Historic average delay times by flight number
avg_flight_delays_by_flight_number = database.query("""
                                SELECT  op_carrier_fl_num, 
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay"
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY op_carrier_fl_num;
                                """)
avg_flight_delays_by_flight_number.to_csv('../../data/preprocessing/average_flight_delays_by_flight_number.csv', index=False)
avg_flight_delays_by_flight_number.head(1)


,op_carrier_fl_num,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay
0,1,29.794118,4.382353,14.529412,3.941176


In [22]:
### Average flight delays grouped by destination airport ID
avg_flight_delays_by_destination_airport = database.query("""
                                SELECT  dest_airport_id, 
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay"
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY dest_airport_id;
                                """)
avg_flight_delays_by_destination_airport.to_csv('../../data/preprocessing/avg_flight_delays_by_destination_airport.csv', index=False)
avg_flight_delays_by_destination_airport.head(1)

,dest_airport_id,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay
0,10135,21.19186,6.401163,9.598837,26.819767


In [23]:
### Average flight delays grouped by origin airport ID
avg_flight_delays_by_origin_airport = database.query("""
                                SELECT  origin_airport_id, 
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay"
                                    FROM flights
                                    WHERE SUBSTRING(fl_date,6,2) = '01'
                                    GROUP BY origin_airport_id;
                                """)
avg_flight_delays_by_origin_airport.to_csv('../../data/preprocessing/avg_flight_delays_by_origin_airport.csv', index=False)
avg_flight_delays_by_origin_airport.head(1)

,origin_airport_id,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay
0,10135,29.207101,2.35503,19.940828,28.378698
